# Open AIs Agent Framework

https://openai.com/index/new-tools-for-building-agents/

## Sessions

In [5]:
from agents import set_trace_processors
set_trace_processors([]) # disable trace processors for this notebook

In [ ]:

import os
from agents import Agent, OpenAIChatCompletionsModel, Runner, SQLiteSession
from openai import AsyncOpenAI, AsyncAzureOpenAI


endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
api_key = os.environ["AZURE_OPENAI_API_KEY"]
deployment = os.environ["AZURE_OPENAI_DEPLOYMENT"]

client = AsyncAzureOpenAI(
    azure_endpoint=endpoint,
    api_key=api_key,
    api_version="2024-08-01-preview")

model = OpenAIChatCompletionsModel( 
    model=deployment,
    openai_client=client
)


session_id = "conversation_123"
session = SQLiteSession(session_id)

haiku_agent = Agent(
    name="Assistant",
    instructions="You only respond in haikus.",
    model=model
)

result = await Runner.run(haiku_agent, "Tell me about recursion in programming.", session=session)
print(result.final_output)



Functions call back,  
Base case halts the cycle,  
Loops in elegance.  


In [7]:
result = await Runner.run(haiku_agent, "Translate this into german.", session=session)
print(result.final_output)


Im Code ruft sich,  
Eine Funktion selbst an,  
Schleifen durch die Zeit.  


## Tool Calls

In [10]:
from pydantic import BaseModel

from agents import Agent, Runner, function_tool


class Weather(BaseModel):
    city: str
    temperature_range: str
    conditions: str


@function_tool
def get_weather(city: str) -> Weather:
    '''Get the current weather for a given city'''

    print(f"[debug] get_weather called for city: {city}")
    return Weather(city=city, temperature_range="14-20C", conditions="Sunny with wind.")


haiku_agent = Agent(
    name="Assistant",
    instructions="You only respond in haikus.",
    model=model,
    tools=[get_weather]
)

result = await Runner.run(haiku_agent, input="What's the weather in Tokyo?")
print(result.final_output)


[debug] get_weather called for city: Tokyo
Tokyo’s weather bright,  
Sunny skies, winds take their flight,  
Fourteen to twenty.


In [12]:
result = await Runner.run(haiku_agent, input="Was ist die Hauptstadt von Deutschland und wie ist da das wetter?")
print(result.final_output)

[debug] get_weather called for city: Berlin
Berlin, Hauptstadt,  
Wetter sonnig und warm,  
Winde wehen sanft.
